# GERMASum: GERman Mean Abstractive Summarization
## Abstractive Summarization of German News articles with Averaging LSTM States
### Training the SentenceClusterModel

In [ ]:
#!pip uninstall -y torch
#!pip install torch==1.8.2+cpu torchvision==0.9.2+cpu torchaudio==0.8.2 torchtext==0.9.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
#!pip install -q cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl
!pip install transformers datasets rouge langdetect sentence-transformers

import torch.nn.functional as F
import pandas as pd
import os
import numpy as np
import torch
import nltk
import time
import json
import pickle

from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from math import log
from tqdm import tqdm
from rouge import Rouge
from datetime import datetime, timezone
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from langdetect import detect

import sys
sys.path.append('/content/modules')

from modules.models import SummaryModel, ModelConfig
from modules.datasets import SentenceClusterDataset, DocSplitDataset
from modules.training import SummaryTrainer

nltk.download('all')

ModuleNotFoundError: ignored

In [ ]:
from models import SummaryModel, ModelConfig
from datasets import SentenceClusterDataset, DocSplitDataset
from training import SummaryTrainer

sys.modules['modules.models'] = sys.modules['models']
sys.modules['modules.datasets'] = sys.modules['datasets']
sys.modules['modules.training'] = sys.modules['training']

In [ ]:
#import os
#assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [ ]:
#import torch_xla
#import torch_xla.core.xla_model as xm

#dev = xm.xla_device()

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/474k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Split Content into sentences

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/export-articles-de-2021-09-08T14:45:27.515899+00:00')
data = data[['CONTENT']].dropna()
print(f'All records: {len(data)}')

All records: 77381


In [ ]:
d = data.iloc[:30000]
dataset = SentenceClusterDataset(tokenizer, d)

/content/modules/datasets.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lambda x: nltk.sent_tokenize(
/content/modules/datasets.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lambda x: [


Downloading:   0%|          | 0.00/968 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/645 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

/content/modules/datasets.py:107: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  clustering_model.fit(sent_emb)
/content/modules/datasets.py:107: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (11). Possibly due to duplicate points in X.
  clustering_model.fit(sent_emb)
/content/modules/datasets.py:107: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  clustering_model.fit(sent_emb)
/content/modules/datasets.py:107: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  clustering_model.fit(sent_emb)
/content/modules/datasets.py:107: ConvergenceWarning: Number of distinct clusters (4) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  clustering_model.fit(sent_emb)
/content/modules/datasets.py:107: 

In [ ]:
test_size = int(len(dataset) * 0.1)
train_size = len(dataset) - test_size
train_set, test_set = random_split(dataset, [train_size, test_size])
print(len(train_set))

with open('/content/drive/MyDrive/datasets/sent-cluster/train_set-2.pt', mode='wb') as f:
  pickle.dump(train_set, f)

with open('/content/drive/MyDrive/datasets/sent-cluster/test_set-2.pt', mode='wb') as f:
  pickle.dump(test_set, f)


186709


In [ ]:
## Train with reconstruction and similarity loss

encoder_config = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm', # lstm, bert_pooled
    #"sequence_encoding_cell_state": 'bert_pooled',
    "embeddings": {
        "mode": "bert_weights_extracted", # bert_weights_extracted, bert
        "trainable": True,
        "dropout": 0.1
    },
    "lstm": {
        "layers": 1,
        "bidirectional": True,
        "dropout": 0,
        "hidden_dim": 512
    }
})

decoder_config = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "final_dropout": 0.5,
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    },
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 1024
    }
})
model = SummaryModel(encoder_config=encoder_config, decoder_config= decoder_config)
model.to('cuda:0')

criterion1 = torch.nn.CrossEntropyLoss(ignore_index=0)
criterion2 = torch.nn.CosineSimilarity(dim=-1)
criterions = {
    'reconstruction_loss': criterion1,
    'similarity_loss': criterion2
}
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

trainer = SummaryTrainer(model, optimizer, criterions, epochs=15, 
                         checkpoint_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster', 
                         tensorboard_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster/metrics',
                         tokenizer=tokenizer,
                         fp16=True)

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda


In [ ]:
trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=0)

Running Epoch 1 ...


Epoch 1: 100%|██████████| 186709/186709 [6:13:28<00:00,  8.33steps/s, loss=3.39]



Epoch Loss: 3.393346777837933
Epoch Time Elapsed: 22408.011373996735


100%|██████████| 20745/20745 [34:09<00:00, 10.12steps/s, val_loss=2.18]



Validation Loss: 2.1828182897944752
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.5002109160912944, 'p': 0.5255124385057967, 'f': 0.5113374191282323}, 'rouge-2': {'r': 0.2766773792593172, 'p': 0.28442377681603154, 'f': 0.27979444733508085}, 'rouge-l': {'r': 0.4661356722197883, 'p': 0.48939396466372753, 'f': 0.4763427198908763}}
Summary Rouge Scores: {'rouge-1': {'r': 0.1999366169240141, 'p': 0.19901218174998198, 'f': 0.19524848361472386}, 'rouge-2': {'r': 0.0553385215268279, 'p': 0.05585554327877885, 'f': 0.05495825589683098}, 'rouge-l': {'r': 0.17715907343124143, 'p': 0.17522504438690603, 'f': 0.17246682715214287}}
Running Epoch 2 ...


...

Epoch 3: 100%|██████████| 186709/186709 [6:30:19<00:00,  7.97steps/s, loss=1.99]



Epoch Loss: 1.9926203569633578
Epoch Time Elapsed: 23419.89708185196


100%|██████████| 20745/20745 [33:50<00:00, 10.22steps/s, val_loss=1.52]



Validation Loss: 1.517663588109476
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.583964500883678, 'p': 0.604993413598196, 'f': 0.5933051073557198}, 'rouge-2': {'r': 0.37256626471410886, 'p': 0.38525087131189906, 'f': 0.3781752198179053}, 'rouge-l': {'r': 0.5583057918289547, 'p': 0.5783409770818937, 'f': 0.5671973590591041}}
Summary Rouge Scores: {'rouge-1': {'r': 0.2135858386570789, 'p': 0.2112394234611426, 'f': 0.20820837741962725}, 'rouge-2': {'r': 0.0696085721813516, 'p': 0.07004002831684804, 'f': 0.06924195814407547}, 'rouge-l': {'r': 0.19017962059185758, 'p': 0.1870983848174144, 'f': 0.18493690520180767}}
Running Epoch 4 ...


Epoch 4:  27%|██▋       | 50707/186709 [1:45:33<4:54:09,  7.71steps/s, loss=1.86]

In [ ]:
with open('/content/drive/MyDrive/datasets/sent-cluster/train_set-2.pt', mode='rb') as f:
  train_set = pickle.load(f)

with open('/content/drive/MyDrive/datasets/sent-cluster/test_set-2.pt', mode='rb') as f:
  test_set = pickle.load(f)

In [ ]:
len(test_set)+len(train_set)

207454

In [ ]:
encoder_config

{'embedding_dim': 768,
 'embeddings': {'dropout': 0.1,
  'mode': 'bert_weights_extracted',
  'trainable': True},
 'hidden_dim': 768,
 'lstm': {'bidirectional': True, 'dropout': 0, 'hidden_dim': 512, 'layers': 1},
 'max_sequence_length': 64,
 'sequence_encoding': 'lstm',
 'vocab_size': 30000}

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/checkpoints/germasum/sent-cluster/checkpoint-epoch-6')
encoder_config = ModelConfig(checkpoint['encoder_config'])
decoder_config = ModelConfig(checkpoint['decoder_config'])
try: 
  model
except NameError:
  model = SummaryModel(encoder_config=encoder_config, decoder_config=decoder_config)
model.load_state_dict(checkpoint['model_state_dict'])
model.to('cuda:0')
try:
  optimizer
except NameError:
  optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

criterion1 = torch.nn.CrossEntropyLoss(ignore_index=0)
criterion2 = torch.nn.CosineSimilarity(dim=-1)
criterions = {
    'reconstruction_loss': criterion1,
    'similarity_loss': criterion2
}

trainer = SummaryTrainer(model, optimizer, criterions, epochs=3, 
                         checkpoint_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster', 
                         tensorboard_path='/content/drive/MyDrive/checkpoints/germasum/sent-cluster/metrics',
                         tokenizer=tokenizer,
                         fp16=True)


Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda


In [28]:
trainer.train(train_set, test_set, eval=True, generation_size=16, from_epoch=6)

Running Epoch 7 ...


Epoch 7: 100%|██████████| 186709/186709 [6:16:02<00:00,  8.28steps/s, loss=1.45]



Epoch Loss: 1.4525033456875283
Epoch Time Elapsed: 22562.681582212448


100%|██████████| 20745/20745 [31:56<00:00, 10.82steps/s, val_loss=1.07]



Validation Loss: 1.0679988432586711
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.6589951440055808, 'p': 0.6767022525415551, 'f': 0.6668051251516129}, 'rouge-2': {'r': 0.4549806500460631, 'p': 0.46421985908899166, 'f': 0.4589815354580439}, 'rouge-l': {'r': 0.6319952699368303, 'p': 0.6485110425674095, 'f': 0.6392606850097279}}
Summary Rouge Scores: {'rouge-1': {'r': 0.21599087243283904, 'p': 0.2226571992465758, 'f': 0.2147358032530274}, 'rouge-2': {'r': 0.07722894307396211, 'p': 0.0778806504316884, 'f': 0.0770385030933809}, 'rouge-l': {'r': 0.19294295788981616, 'p': 0.19752565828697882, 'f': 0.1912653743381959}}
Running Epoch 8 ...


...


Validation Loss: 0.9961588509312352
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.6886040039987338, 'p': 0.694005404588918, 'f': 0.6908219835628033}, 'rouge-2': {'r': 0.512741223414433, 'p': 0.5174987399076214, 'f': 0.5147956842110121}, 'rouge-l': {'r': 0.6658061221098589, 'p': 0.6709649074901822, 'f': 0.6679246140161448}}
Summary Rouge Scores: {'rouge-1': {'r': 0.21574357481693152, 'p': 0.22501526229467317, 'f': 0.21552552718132023}, 'rouge-2': {'r': 0.07956327562217985, 'p': 0.08065495188610039, 'f': 0.07956708414075313}, 'rouge-l': {'r': 0.19348115207610994, 'p': 0.20045777856177208, 'f': 0.19276208437977918}}
Running Epoch 9 ...


Epoch 9: 100%|██████████| 186709/186709 [6:16:14<00:00,  8.27steps/s, loss=1.32]



Epoch Loss: 1.31511909395755
Epoch Time Elapsed: 22574.80316901207


100%|██████████| 20745/20745 [32:34<00:00, 10.62steps/s, val_loss=0.95]



Validation Loss: 0.9498308301760412
Reconstruction Rouge Scores: {'rouge-1': {'r': 0.6970556032706627, 'p': 0.7178898370702173, 'f': 0.7065367629631416}, 'rouge-2': {'r': 0.5272581119205034, 'p': 0.5368101008887572, 'f': 0.531565430760359}, 'rouge-l': {'r': 0.6754078333164909, 'p': 0.6952187500305768, 'f': 0.6844152105331763}}
Summary Rouge Scores: {'rouge-1': {'r': 0.21783909880524388, 'p': 0.2245143376909795, 'f': 0.21653321767864453}, 'rouge-2': {'r': 0.08197921521497234, 'p': 0.08289523147288083, 'f': 0.0818674385851764}, 'rouge-l': {'r': 0.1953655784911655, 'p': 0.2001140258921683, 'f': 0.19371423286878536}}
Total Time Elapsed: 73665.3167898655


In [29]:
test = {
    'input_ids': train_set.dataset.X_enc[0]['input_ids'][0:1]
}
print(tokenizer.decode(test['input_ids'][0], skip_special_tokens=True))
with torch.no_grad():
  generated = model.reconstruct(test, n_beams=5, return_all_candidates=False)
  decoded = tokenizer.decode(generated[0][0], skip_special_tokens=True)
  print(decoded)
  #rouge = Rouge()
  #ref = tokenizer.decode(inp['input_ids'][0], skip_special_tokens=True)
  #scores = rouge.get_scores(test_sentence, decoded, avg=True)

Nach regierungsinternen Diskussionen sei das aber revidiert worden.
Nach regierungsinternen Diskussionen sei das aber revidiert worden.


In [30]:
test_summary = train_set.dataset.X_enc[1]
print(tokenizer.batch_decode(test_summary['input_ids'], skip_special_tokens=True))
with torch.no_grad():
  generated = model.summarize_sequences(test_summary)
  print(tokenizer.decode(generated[0][0], skip_special_tokens=True))

['Er habe sich aber dazu entschlossen, verhältnismäßig zu reagieren. Biden hatte Kremlchef Wladimir Putin in einem Telefonat am Dienstag ein Gipfeltreffen in einem Drittland vorgeschlagen.', 'Ob Putin die Einladung annimmt, ist weiter offen.', 'Im Kreml hieß es dazu, dass Sanktionen für solche Gipfelpläne nicht hilfreich seien.']
Im Kreml gab der Regierung, lang Falte sich dennoch habe dann weiter : ein mit vermeintlich Wladimir Putin.


In [ ]:
generated

(tensor([[    3,   482,   350,    30,  3693,  8480,  2241,    30,  4458,   188,
           4267, 26935,  6325,   204, 11242, 26911,  4090, 26914,     4,     4]],
        device='cuda:0'), 20.766192579122016)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

### Training the DocSplitModel 
Split Documents into n(=4) sub-parts and train summary on those

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/export-articles-de-2021-09-08T14:45:27.515899+00:00')
#data = data[['CONTENT']].dropna()
# take only first 5000 articles
data = data.iloc[:30000]

In [ ]:
dataset = DocSplitDataset(tokenizer, data)

In [ ]:
## Train with reconstruction and similarity loss

encoder_config = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm', # lstm, bert_pooled
    #"sequence_encoding_cell_state": 'bert_pooled',
    "embeddings": {
        "mode": "bert_weights_extracted", # bert_weights_extracted, bert
        "trainable": True,
        "dropout": 0.1
    },
    "lstm": {
        "layers": 1,
        "bidirectional": True,
        "dropout": 0,
        "hidden_dim": 512
    }
})

decoder_config = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "final_dropout": 0.5,
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": True,
        "dropout": 0.1
    },
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 1024
    }
})
model = SummaryModel(encoder_config=encoder_config, decoder_config= decoder_config)
model.to('cuda:0')

criterion1 = torch.nn.CrossEntropyLoss(ignore_index=0)
criterion2 = torch.nn.CosineSimilarity(dim=-1)
criterions = {
    'reconstruction_loss': criterion1,
    'similarity_loss': criterion2
}
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

trainer = SummaryTrainer(model, optimizer, criterions, epochs=4, 
                         checkpoint_path='/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-4', 
                         tensorboard_path='/content/drive/MyDrive/runs',
                         tokenizer=tokenizer,
                         fp16=True)

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda


In [ ]:
test_size = int(len(dataset) * 0.1)
train_size = len(dataset) - test_size
train_set, test_set = random_split(dataset, [train_size, test_size])
print(len(train_set))

with open('/content/drive/MyDrive/datasets/doc-split/train_set-2.pt', mode='wb') as f:
  pickle.dump(train_set, f)

with open('/content/drive/MyDrive/datasets/doc-split/test_set-2.pt', mode='wb') as f:
  pickle.dump(test_set, f)

26150


In [ ]:
trainer.train(train_set, test_set, eval=True, generation_size=32, from_epoch=0)

Running Epoch 1 ...


Epoch 1: 100%|██████████| 26150/26150 [7:39:51<00:00,  1.06s/steps, loss=4.73]



Epoch Loss: 4.7305673095857435
Epoch Time Elapsed: 27591.78464460373


100%|██████████| 2905/2905 [51:38<00:00,  1.07s/steps, val_loss=3.92]



Validation Loss: 3.919837644252827
Rouge Scores: {'rouge-1': {'r': 0.3166369617322374, 'p': 0.17914816776788217, 'f': 0.21615398664013216}, 'rouge-2': {'r': 0.0776315336593444, 'p': 0.05181684141344359, 'f': 0.058119195074027574}, 'rouge-l': {'r': 0.29804285118671914, 'p': 0.16755867307085923, 'f': 0.2030863797536572}}
Summary Rouge Scores: {'rouge-1': {'r': 0.26701504483143157, 'p': 0.14165953883347437, 'f': 0.17569920960566185}, 'rouge-2': {'r': 0.03455558991476699, 'p': 0.018980894549399066, 'f': 0.0228804369555645}, 'rouge-l': {'r': 0.24511605208213028, 'p': 0.13003552195179838, 'f': 0.16119922910726273}}
Running Epoch 2 ...


Epoch 2: 100%|██████████| 26150/26150 [7:46:21<00:00,  1.07s/steps, loss=4.02]



Epoch Loss: 4.0239081680816
Epoch Time Elapsed: 27981.39450263977


100%|██████████| 2905/2905 [50:54<00:00,  1.05s/steps, val_loss=3.64]



Validation Loss: 3.641436288752921
Rouge Scores: {'rouge-1': {'r': 0.3471180279237308, 'p': 0.21094787822294922, 'f': 0.24924869738055816}, 'rouge-2': {'r': 0.099118246416245, 'p': 0.07047453802817699, 'f': 0.07755666364475682}, 'rouge-l': {'r': 0.327425025377651, 'p': 0.19892175400721152, 'f': 0.23487516059473074}}
Summary Rouge Scores: {'rouge-1': {'r': 0.2700024787455477, 'p': 0.15493735322804908, 'f': 0.18655349030407503}, 'rouge-2': {'r': 0.03771546902917915, 'p': 0.022240918293997027, 'f': 0.02613624119300399}, 'rouge-l': {'r': 0.24657958437772357, 'p': 0.1414226068681972, 'f': 0.17019898782156703}}
Running Epoch 3 ...


Epoch 3:  85%|████████▌ | 22245/26150 [6:32:21<1:16:15,  1.17s/steps, loss=3.83]

In [ ]:
checkpoint = torch.load('/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-4/checkpoint-epoch-6')
encoder_config = ModelConfig(checkpoint['encoder_config'])
decoder_config = ModelConfig(checkpoint['decoder_config'])
try: 
  model
except NameError:
  model = SummaryModel(encoder_config=encoder_config, decoder_config=decoder_config)
model.load_state_dict(checkpoint['model_state_dict'])
model.to('cuda:0')
try:
  optimizer
except NameError:
  optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

criterion1 = torch.nn.CrossEntropyLoss(ignore_index=0)
criterion2 = torch.nn.CosineSimilarity(dim=-1)
criterions = {
    'reconstruction_loss': criterion1,
    'similarity_loss': criterion2
}

trainer = SummaryTrainer(model, optimizer, criterions, epochs=2, 
                         checkpoint_path='/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-4', 
                         tensorboard_path='/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-4/metrics',
                         tokenizer=tokenizer,
                         fp16=True)

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Using device: cuda


In [ ]:
with open('/content/drive/MyDrive/datasets/doc-split/train_set-2.pt', mode='rb') as f:
  train_set = pickle.load(f)

with open('/content/drive/MyDrive/datasets/doc-split/test_set-2.pt', mode='rb') as f:
  test_set = pickle.load(f)

In [ ]:
trainer.train(train_set, test_set, eval=True, generation_size=32, from_epoch=6)

Running Epoch 7 ...


Epoch 7: 100%|██████████| 26150/26150 [7:43:40<00:00,  1.06s/steps, loss=3.51]



Epoch Loss: 3.513296066947837
Epoch Time Elapsed: 27820.069763183594


100%|██████████| 2905/2905 [51:08<00:00,  1.06s/steps, val_loss=3.25]



Validation Loss: 3.2475408328176383
Rouge Scores: {'rouge-1': {'r': 0.39884627626200003, 'p': 0.27299461099875627, 'f': 0.3104760125554185}, 'rouge-2': {'r': 0.14947930085647698, 'p': 0.11316435851070071, 'f': 0.12227603711666014}, 'rouge-l': {'r': 0.3759568898706281, 'p': 0.2575603190545017, 'f': 0.2925998231589457}}
Summary Rouge Scores: {'rouge-1': {'r': 0.27823172219694536, 'p': 0.17582801489775313, 'f': 0.20497415246422604}, 'rouge-2': {'r': 0.03951560380794148, 'p': 0.026157040627729045, 'f': 0.02949598176214541}, 'rouge-l': {'r': 0.25205282275505536, 'p': 0.1590677049799588, 'f': 0.185427064437369}}
Running Epoch 8 ...


Epoch 8: 100%|██████████| 26150/26150 [7:44:03<00:00,  1.06s/steps, loss=3.47]



Epoch Loss: 3.467924015433394
Epoch Time Elapsed: 27843.16139650345


100%|██████████| 2905/2905 [51:08<00:00,  1.06s/steps, val_loss=3.21]



Validation Loss: 3.20629877881651
Rouge Scores: {'rouge-1': {'r': 0.40621156084584986, 'p': 0.2793062887309845, 'f': 0.31910413313419195}, 'rouge-2': {'r': 0.1585779506963309, 'p': 0.12007021413516192, 'f': 0.13088965793177715}, 'rouge-l': {'r': 0.37914069630593605, 'p': 0.26187057425365745, 'f': 0.2984654705360201}}
Summary Rouge Scores: {'rouge-1': {'r': 0.2796421884606189, 'p': 0.17723157173047357, 'f': 0.20627060082199955}, 'rouge-2': {'r': 0.04031700335680554, 'p': 0.02654222590854046, 'f': 0.0300914626980606}, 'rouge-l': {'r': 0.25377045126244063, 'p': 0.16055749616376552, 'f': 0.18690630412753462}}
Total Time Elapsed: 62063.13387441635


# Use pre-trained bert-2-bert embedding matrix

In [ ]:
from transformers import EncoderDecoderModel

bert2bert = EncoderDecoderModel.from_pretrained('./drive/MyDrive/checkpoints/bert2bert/reconstruct-content/v3/checkpoint-4500')

encoder_bert = bert2bert.encoder
decoder_bert = bert2bert.decoder.bert

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/export-articles-de-2021-09-08T14:45:27.515899+00:00')
data = data[['CONTENT']].dropna()
# take only first 5000 articles
data = data.iloc[:30000]

In [ ]:
with open('/content/drive/MyDrive/datasets/doc-split/train_set-2.pt', mode='rb') as f:
  train_set = pickle.load(f)

with open('/content/drive/MyDrive/datasets/doc-split/test_set-2.pt', mode='rb') as f:
  test_set = pickle.load(f)

In [ ]:
encoder_config = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "sequence_encoding": 'lstm', # lstm, bert_pooled
    #"sequence_encoding_cell_state": 'bert_pooled',
    "embeddings": {
        "mode": "bert_weights_extracted", # bert_weights_extracted, bert
        "trainable": False,
        "dropout": 0.1
    },
    "lstm": {
        "layers": 1,
        "bidirectional": True,
        "dropout": 0,
        "hidden_dim": 512
    }
})

decoder_config = ModelConfig({
    "embedding_dim": 768,
    "max_sequence_length": 64,
    "vocab_size": 30000,
    "final_dropout": 0.5,
    "embeddings": {
        "mode": "bert_weights_extracted",
        "trainable": False,
        "dropout": 0.1
    },
    "lstm": {
        "layers": 1,
        "bidirectional": False,
        "dropout": 0,
        "hidden_dim": 1024
    }
})
model = SummaryModel(encoder_config=encoder_config, decoder_config= decoder_config, bert_models=(encoder_bert, decoder_bert))
model.to('cuda:0')

criterion1 = torch.nn.CrossEntropyLoss(ignore_index=0)
criterion2 = torch.nn.CosineSimilarity(dim=-1)
criterions = {
    'reconstruction_loss': criterion1,
    'similarity_loss': criterion2
}
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

trainer = SummaryTrainer(model, optimizer, criterions, epochs=4, 
                         checkpoint_path='/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-5', 
                         tensorboard_path='/content/drive/MyDrive/checkpoints/encoder-decoder-lstm/sum-model/exp-5/metrics',
                         tokenizer=tokenizer,
                         fp16=True)

Using device: cuda


In [ ]:
trainer.train(train_set, test_set, eval=True, generation_size=32, from_epoch=0)

Running Epoch 1 ...


Epoch 1: 100%|██████████| 26150/26150 [10:14:26<00:00,  1.41s/steps, loss=4.82]



Epoch Loss: 4.824425172658769
Epoch Time Elapsed: 36866.658673763275


100%|██████████| 2905/2905 [1:10:46<00:00,  1.46s/steps, val_loss=3.97]



Validation Loss: 3.9749043764675203
Rouge Scores: {'rouge-1': {'r': 0.33433560546220586, 'p': 0.17268660940385921, 'f': 0.2167767543816698}, 'rouge-2': {'r': 0.08045219590697052, 'p': 0.046409035513348594, 'f': 0.05533405488571573}, 'rouge-l': {'r': 0.3151299995120059, 'p': 0.1629240533965102, 'f': 0.20437772917775585}}
Summary Rouge Scores: {'rouge-1': {'r': 0.2776484853539511, 'p': 0.13576149410294575, 'f': 0.17274788841785202}, 'rouge-2': {'r': 0.037098675465559255, 'p': 0.01876346953525056, 'f': 0.023265123821502146}, 'rouge-l': {'r': 0.2561096911138283, 'p': 0.12519215801562508, 'f': 0.15920925764330476}}
Running Epoch 2 ...


Epoch 2: 100%|██████████| 26150/26150 [10:52:04<00:00,  1.50s/steps, loss=4.07]



Epoch Loss: 4.073721865387224
Epoch Time Elapsed: 39124.583790779114


100%|██████████| 2905/2905 [1:11:52<00:00,  1.48s/steps, val_loss=3.68]


In [ ]:
!pip freeze | grep scikit

scikit-image==0.16.2
scikit-learn==0.22.2.post1
